In [1]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from link_prediction import *

In [4]:
#Load the entity embeddings dict
with open("nguyen_fb15k237_transe_entity_embeddings.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("nguyen_fb15k237_transe_predicate_embeddings.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

train_triples = pd.read_csv('fb15k237_train.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])
test_triples = pd.read_csv('fb15k237_test.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])

**Create Negative Samples**

In [5]:
training_object=Training(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE')
tail_df = training_object.create_training_data_filtered(n=1, creating_for="tail")
head_df = training_object.create_training_data_filtered(n=1, creating_for="head")
print(len(tail_df))
print(len(head_df))

Processing triples: 100%|██████████| 272115/272115 [02:11<00:00, 2070.31triple/s]

93371
56317


**Prepare Training Data**

In [6]:
train_triples['label']=1
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: training_object.get_embedding(row).cpu().numpy(), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)
train_df['label'].value_counts()

,count
label,
1,272115
0,146168


**Set Models**

**MLP**

In [7]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [8]:
param_grid_mlp = {
    'hidden_layer_sizes': [ (64, 64), (128, 128), (256, 256)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01],
    'max_iter': [50, 100]
}



results_mlp = []
for hidden_layer_sizes in param_grid_mlp['hidden_layer_sizes']:
    for alpha in param_grid_mlp['alpha']:
        for learning_rate_init in param_grid_mlp['learning_rate_init']:
            for max_iter in param_grid_mlp['max_iter']:
                mlp = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    alpha=alpha,
                    learning_rate_init=learning_rate_init,
                    max_iter=max_iter,
                    solver='adam',
                    early_stopping=True,
                    batch_size=32,
                    random_state=42
                )
                mlp.fit(X_train, y_train)
                preds = mlp.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_mlp.append({
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'alpha': alpha,
                    'learning_rate_init': learning_rate_init,
                    'max_iter': max_iter,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_mlp = pd.DataFrame(results_mlp)

best_mlp_params = df_results_mlp.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for MLP:", best_mlp_params)

Best parameters for MLP: {'hidden_layer_sizes': (256, 256), 'alpha': 0.0001, 'learning_rate_init': 0.001, 'max_iter': 100, 'f1_score': 0.8470329748214823, 'accuracy': 0.7936120873078486, 'precision_0': 0.7373849571564582, 'precision_1': 0.8178614200171086, 'recall_0': 0.6358349866593692, 'recall_1': 0.8783624871380273}


**XGB**

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'scale_pos_weight': [0.5, 1],
    'learning_rate': [0.01, 0.1, 0.2]
}

results_xgb = []
for n_estimators in param_grid_xgb['n_estimators']:
    for max_depth in param_grid_xgb['max_depth']:
        for scale_pos_weight in param_grid_xgb['scale_pos_weight']:
            for learning_rate in param_grid_xgb['learning_rate']:
                xgb = XGBClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    scale_pos_weight=scale_pos_weight,
                    learning_rate=learning_rate,
                    random_state=42,
                    use_label_encoder=False,
                    eval_metric='logloss'
                )
                xgb.fit(X_train, y_train)
                preds = xgb.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_xgb.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'scale_pos_weight': scale_pos_weight,
                    'learning_rate': learning_rate,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_xgb = pd.DataFrame(results_xgb)
best_xgb_params = df_results_xgb.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for XGBClassifier:", best_xgb_params)

**LGBM**

In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2],
    'is_unbalance': [True, False]
}

results_lgbm = []
for n_estimators in param_grid_lgbm['n_estimators']:
    for max_depth in param_grid_lgbm['max_depth']:
        for learning_rate in param_grid_lgbm['learning_rate']:
            for is_unbalance in param_grid_lgbm['is_unbalance']:
                lgbm = LGBMClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    is_unbalance=is_unbalance,
                    random_state=42
                )
                lgbm.fit(X_train, y_train)
                preds = lgbm.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_lgbm.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'is_unbalance': is_unbalance,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_lgbm = pd.DataFrame(results_lgbm)
best_lgbm_params = df_results_lgbm.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for LGBM:", best_lgbm_params)

**Train Ensemble**

In [11]:
# Best MLPClassifier
best_mlp = MLPClassifier(
    hidden_layer_sizes=best_mlp_params['hidden_layer_sizes'],
    alpha=best_mlp_params['alpha'],
    learning_rate_init=best_mlp_params['learning_rate_init'],
    max_iter=best_mlp_params['max_iter'],
    solver='adam',
    early_stopping=True,
    batch_size=32,
    random_state=42
)
best_mlp.fit(X_train, y_train)
preds_mlp = best_mlp.predict(X_test)
report_mlp = classification_report(y_test, preds_mlp)

best_xgb = XGBClassifier(
    n_estimators=int(best_xgb_params['n_estimators']),
    max_depth=int(best_xgb_params['max_depth']),
    learning_rate=float(best_xgb_params['learning_rate']),
    scale_pos_weight=float(best_xgb_params['scale_pos_weight']),
    random_state=42,
)
best_xgb.fit(X_train, y_train)
preds_xgb = best_xgb.predict(X_test)
report_xgb = classification_report(y_test, preds_xgb)

# Best LGBMClassifier
best_lgbm = LGBMClassifier(
    n_estimators=best_lgbm_params['n_estimators'],
    max_depth=best_lgbm_params['max_depth'],
    learning_rate=best_lgbm_params['learning_rate'],
    is_unbalance=best_lgbm_params['is_unbalance'],
    random_state=42
)
best_lgbm.fit(X_train, y_train)
preds_lgbm = best_lgbm.predict(X_test)
report_lgbm = classification_report(y_test, preds_lgbm)

######################################
# Ensemble the best models using VotingClassifier
######################################
ensemble_model = VotingClassifier(
    estimators=[
        ('mlp', best_mlp),
        ('xgb', best_xgb),
        ('lgbm', best_lgbm)
    ],
    voting='soft',  # Soft voting uses predicted probabilities
    n_jobs=-1
)

start_time = time.time()
ensemble_model.fit(X_train, y_train)
end_time = time.time()

ensemble_time = end_time - start_time
preds_ensemble = ensemble_model.predict(X_test)
ensemble_acc = accuracy_score(y_test, preds_ensemble)
report_ensemble = classification_report(y_test, preds_ensemble)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 244903, number of negative: 131551
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.322060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73922
[LightGBM] [Info] Number of data points in the train set: 376454, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.650552 -> initscore=0.621468
[LightGBM] [Info] Start training from score 0.621468


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [12]:
y_pred_ensemble = ensemble_model.predict(X_test)
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Ensemble Classification Report:\n", classification_report(y_test, y_pred_ensemble))

Ensemble Accuracy: 0.804752683544909
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.64      0.70     14617
           1       0.82      0.89      0.86     27212

    accuracy                           0.80     41829
   macro avg       0.79      0.77      0.78     41829
weighted avg       0.80      0.80      0.80     41829



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
with open("/content/drive/MyDrive/link_prediction/tuning/fb15k237_transe_tuning.txt", "w") as f:
    f.write("##############################\n")
    f.write("MLPClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_mlp.to_string())
    f.write("\n\nBest Parameters for MLPClassifier:\n")
    f.write(str(best_mlp_params))
    f.write("\n\nClassification Report for Best MLPClassifier:\n")
    f.write(report_mlp)

    f.write("\n\n\n##############################\n")
    f.write("XGBClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_xgb.to_string())
    f.write("\n\nBest Parameters for XGBClassifier:\n")
    f.write(str(best_xgb_params))
    f.write("\n\nClassification Report for Best XGBClassifier:\n")
    f.write(report_xgb)

    f.write("\n\n\n##############################\n")
    f.write("LGBMClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_lgbm.to_string())
    f.write("\n\nBest Parameters for LGBMClassifier:\n")
    f.write(str(best_lgbm_params))
    f.write("\n\nClassification Report for Best LGBMClassifier:\n")
    f.write(report_lgbm)

    f.write("\n\n\n##############################\n")
    f.write("Ensemble Model Performance\n")
    f.write("##############################\n")
    f.write("Ensemble Accuracy: {:.4f}\n".format(ensemble_acc))
    f.write("Ensemble Classification Report:\n")
    f.write(report_ensemble)

print("Results saved to model_results.txt")

Results saved to model_results.txt


**Rerank Hitk**

In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='TransE', k=50)

Precomputing top-k predictions: 100%|██████████| 20466/20466 [01:30<00:00, 227.29it/s]


In [16]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10,20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19,20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 11.93% | tail: 31.18% | overall: 23.50%
k=1 | th=2 | head: 13.36% | tail: 33.25% | overall: 25.31%
k=1 | th=3 | head: 12.68% | tail: 31.64% | overall: 24.07%
k=1 | th=4 | head: 11.48% | tail: 29.64% | overall: 22.40%
k=1 | th=5 | head: 10.88% | tail: 27.84% | overall: 21.07%
k=1 | th=6 | head: 10.52% | tail: 26.35% | overall: 20.04%
k=1 | th=7 | head: 10.13% | tail: 25.19% | overall: 19.18%
k=1 | th=8 | head: 9.74% | tail: 24.19% | overall: 18.42%
k=1 | th=9 | head: 9.33% | tail: 23.55% | overall: 17.88%
k=1 | th=10 | head: 8.99% | tail: 22.76% | overall: 17.26%
k=1 | th=20 | head: 7.06% | tail: 19.35% | overall: 14.45%
k=1 | th=30 | head: 6.01% | tail: 17.65% | overall: 13.01%
k=1 | th=40 | head: 5.55% | tail: 16.75% | overall: 12.28%
k=1 | th=50 | head: 5.05% | tail: 16.16% | overall: 11.73%
k=3 | th=3 | head: 24.30% | tail: 45.69% | overall: 37.16%
k=3 | th=4 | head: 24.12% | tail: 45.21% | overall: 36.80%
k=3 | th=5 | head: 23.20% | tail: 43.72% | overall: 35.53%

In [17]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 11.93% | tail: 31.18% | overall: 23.50%
k=1 | th=2 | head: 15.26% | tail: 35.36% | overall: 27.34%
k=1 | th=3 | head: 13.87% | tail: 33.34% | overall: 25.57%
k=1 | th=4 | head: 12.37% | tail: 30.72% | overall: 23.40%
k=1 | th=5 | head: 11.58% | tail: 28.67% | overall: 21.85%
k=1 | th=6 | head: 11.08% | tail: 26.64% | overall: 20.43%
k=1 | th=7 | head: 10.84% | tail: 25.44% | overall: 19.62%
k=1 | th=8 | head: 10.61% | tail: 24.62% | overall: 19.03%
k=1 | th=9 | head: 10.16% | tail: 24.07% | overall: 18.52%
k=1 | th=10 | head: 9.77% | tail: 23.46% | overall: 17.99%
k=1 | th=20 | head: 8.38% | tail: 20.79% | overall: 15.84%
k=1 | th=30 | head: 7.59% | tail: 19.57% | overall: 14.79%
k=1 | th=40 | head: 7.34% | tail: 18.94% | overall: 14.31%
k=1 | th=50 | head: 7.10% | tail: 18.62% | overall: 14.02%
k=3 | th=3 | head: 24.30% | tail: 45.69% | overall: 37.16%
k=3 | th=4 | head: 25.35% | tail: 46.11% | overall: 37.83%
k=3 | th=5 | head: 24.45% | tail: 44.98% | overall: 36.7

In [18]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 11.93% | tail: 31.18% | overall: 23.50%
k=1 | th=2 | head: 13.23% | tail: 32.82% | overall: 25.00%
k=1 | th=3 | head: 12.48% | tail: 31.08% | overall: 23.66%
k=1 | th=4 | head: 11.26% | tail: 28.91% | overall: 21.87%
k=1 | th=5 | head: 10.62% | tail: 27.27% | overall: 20.63%
k=1 | th=6 | head: 10.16% | tail: 25.71% | overall: 19.50%
k=1 | th=7 | head: 9.51% | tail: 24.63% | overall: 18.60%
k=1 | th=8 | head: 9.06% | tail: 24.05% | overall: 18.07%
k=1 | th=9 | head: 8.80% | tail: 23.69% | overall: 17.75%
k=1 | th=10 | head: 8.61% | tail: 23.24% | overall: 17.40%
k=1 | th=20 | head: 7.29% | tail: 20.37% | overall: 15.15%
k=1 | th=30 | head: 6.44% | tail: 19.10% | overall: 14.05%
k=1 | th=40 | head: 5.88% | tail: 18.36% | overall: 13.38%
k=1 | th=50 | head: 5.69% | tail: 17.77% | overall: 12.95%
k=3 | th=3 | head: 24.30% | tail: 45.69% | overall: 37.16%
k=3 | th=4 | head: 24.07% | tail: 44.63% | overall: 36.42%
k=3 | th=5 | head: 22.93% | tail: 43.29% | overall: 35.16%


In [19]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 11.93% | tail: 31.18% | overall: 23.50%
k=1 | th=2 | head: 14.50% | tail: 34.37% | overall: 26.44%
k=1 | th=3 | head: 13.67% | tail: 32.42% | overall: 24.93%
k=1 | th=4 | head: 12.44% | tail: 30.40% | overall: 23.23%
k=1 | th=5 | head: 11.72% | tail: 28.46% | overall: 21.78%
k=1 | th=6 | head: 11.25% | tail: 27.02% | overall: 20.73%
k=1 | th=7 | head: 10.74% | tail: 26.26% | overall: 20.07%
k=1 | th=8 | head: 10.53% | tail: 25.47% | overall: 19.51%
k=1 | th=9 | head: 10.35% | tail: 24.96% | overall: 19.13%
k=1 | th=10 | head: 9.99% | tail: 24.47% | overall: 18.69%
k=1 | th=20 | head: 8.84% | tail: 22.11% | overall: 16.82%
k=1 | th=30 | head: 8.13% | tail: 21.27% | overall: 16.03%
k=1 | th=40 | head: 7.73% | tail: 20.70% | overall: 15.53%
k=1 | th=50 | head: 7.50% | tail: 20.36% | overall: 15.23%
k=3 | th=3 | head: 24.30% | tail: 45.69% | overall: 37.16%
k=3 | th=4 | head: 24.85% | tail: 45.58% | overall: 37.31%
k=3 | th=5 | head: 24.04% | tail: 44.38% | overall: 36.2